For Collab

In [ ]:
!git clone https://github.com/GerardTho/Deep_image_prior_inpainting.git

In [ ]:
cd Deep_image_prior_inpainting

In [1]:
import torch
import numpy as np
from PIL import Image

import utils
from model.EncoderDecoder import EncoderDecoder
from model.EncoderDecoderSkipConnections import EncoderDecoderSkipConnections
from model.EncoderDecoderResidualConnections import EncoderDecoderResidualConnections
from model.EdgeModeInpainting import InpaintGenerator, EdgeGenerator

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(device)

cpu


In [3]:
INPUT = 'noise'
input_depth = 3
LR = 0.01
num_iter = 5000
param_noise = False
show_every = 100
figsize = 5
reg_noise_std = 0
OPTIMIZER = 'adam'
model_selection = "EncoderDecoderSkipConnections"

In [4]:
img_path  = 'data/vase.png'
mask_path = 'data/vase_mask.png'
img_pil, img_np = utils.get_image(img_path)
img_mask_pil, img_mask_np = utils.get_image(mask_path)
img_mask_pil = utils.crop_image(img_mask_pil)
img_pil      = utils.crop_image(img_pil)
img_np      = utils.pil_to_np(img_pil)
img_mask_np = utils.pil_to_np(img_mask_pil)

In [5]:
if model_selection == "EncoderDecoder":
    model = EncoderDecoder(num_input_channels=input_depth, num_output_channels=img_np.shape[0],
                        n_upsampler=[128, 128, 128, 64, 32, 16],
                        n_downsampler=[16, 32, 64, 128, 128, 128],
                        k_upsampler=[3, 3, 3, 3, 3, 3],
                        k_downsampler=[5, 5, 5, 5, 5, 5],).to(device)
elif model_selection == "EncoderDecoderSkipConnections":
    model = EncoderDecoderSkipConnections(num_input_channels=input_depth, num_output_channels=img_np.shape[0],
                        n_upsampler=[128, 128, 128, 64, 32, 16],
                        n_downsampler=[16, 32, 64, 128, 128, 128],
                        n_skip=[0,0,0,4,4,4],
                        k_upsampler=[3, 3, 3, 3, 3, 3],
                        k_downsampler=[5, 5, 5, 5, 5, 5],
                        k_skip=[1,1,1,1,1,1],).to(device)
elif model_selection == "EncoderDecoderResidualConnections":
    model = EncoderDecoderResidualConnections(num_input_channels=input_depth, num_output_channels=img_np.shape[0],
                        n_upsampler=[128, 128, 128, 64, 32, 16],
                        n_downsampler=[16, 32, 64, 128, 128, 128],
                        k_upsampler=[3, 3, 3, 3, 3, 3],
                        k_downsampler=[5, 5, 5, 5, 5, 5],).to(device)

In [6]:
# Compute number of parameters
s  = sum(np.prod(list(p.size())) for p in model.parameters())
print ('Number of params: %d' % s)

img_var = torch.from_numpy(img_np).reshape(-1, img_np.shape[0], img_np.shape[1], img_np.shape[2]).to(device)
mask_var = torch.from_numpy(img_mask_np).reshape(-1, img_mask_np.shape[0], img_mask_np.shape[1], img_mask_np.shape[2]).to(device)

# Loss
mse = torch.nn.MSELoss()

net_input = utils.generate_noise(input_depth, img_np.shape[1:]).to(device)

Number of params: 3155763


In [ ]:
def optimize(parameters, LR, num_iter):
  optimizer = torch.optim.Adam(parameters, lr=LR)
  for j in range(num_iter):
    optimizer.zero_grad()

    net_input = net_input_saved
    if reg_noise_std > 0:
        net_input = net_input_saved + (noise.normal_() * reg_noise_std)

    out = model(net_input)
    total_loss = mse(out * mask_var, img_var * mask_var)
    total_loss.backward()

    optimizer.step()

    print ('Iteration %05d    Loss %f' % (j, total_loss.item()), '\r', end='')
    if j % show_every == 0:
        out_np = out.detach().cpu().numpy()[0]
        utils.plot_image_grid([np.clip(out_np, 0, 1)], factor=figsize, nrow=1)

net_input_saved = net_input.detach().clone()
noise = net_input.detach().clone()

p = [x for x in model.parameters()]
optimize(p, LR, num_iter)

In [ ]:
out_np = model(net_input).detach().cpu().numpy()[0]
utils.plot_image_grid([out_np], factor=5);

In [ ]:
img_path  = 'data/vase.png'
mask_path = 'data/vase_mask.png'
img_pil, img_np = utils.get_image(img_path)
img_pil.thumbnail((256, 256), Image.Resampling.LANCZOS)
img_mask_pil, img_mask_np = utils.get_image(mask_path)
img_mask_pil.thumbnail((256, 256), Image.Resampling.LANCZOS)
img_mask_pil = utils.crop_image(img_mask_pil)
img_pil      = utils.crop_image(img_pil)
img_np      = utils.pil_to_np(img_pil)
img_mask_np = utils.pil_to_np(img_mask_pil)
img_masked = img_np*img_mask_np + (1-img_mask_np)

In [ ]:
edge_model = EdgeGenerator(init_weights=False)
edge_model.load_state_dict(torch.load('EdgeModel_gen.pth')["generator"])
inpaint_model = InpaintGenerator(init_weights=False)
inpaint_model.load_state_dict(torch.load('InpaintingModel_gen.pth')["generator"])

In [ ]:
img_edge = edge_model(torch.from_numpy(img_masked))

In [ ]:
img_inpainted = inpaint_model(torch.cat((torch.from_numpy(img_masked), img_edge), dim=0)).detach().cpu().numpy()

In [ ]:
utils.plot_image_grid([np.clip(1-img_edge.detach().cpu().numpy(), 0, 1)], factor=figsize, nrow=1)

In [ ]:
utils.plot_image_grid([np.clip(img_inpainted, 0, 1)], factor=figsize, nrow=1)